In [2]:
# 필요한 패키지 불러오기
import pandas as pd
from konlpy.tag import Okt 
okt = Okt()
import gensim
import gensim.corpora as corpora
import os
# os.environ["PYTHONIOENCODING"] = "utf-8”
import pyLDAvis.gensim

In [3]:
# 파일 불러오기
df1 = pd.read_excel('Samsung1.xlsx')
df2 = pd.read_excel('Samsung2.xlsx')
df3 = pd.read_excel('Samsung3.xlsx')
df4 = pd.read_excel('Samsung4.xlsx')
df5 = pd.read_excel('Samsung5.xlsx')

# 열 방향으로 데이터프레임 병합
combined_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

# total data 저장
combined_df.to_excel('ToTSamsung.xlsx', index=False)


In [4]:
# 저장한 데이터 불러오기
df = pd.read_excel('ToTSamsung.xlsx')
df.head()

,상품명,구매자 이름,구매자 평점,리뷰 제목,리뷰 내용,맛 만족도
0,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",이*영,5,"가성비 갑, 성능 좋아요!! 추천드립니다!",**내돈내산 후기입니다**구매 및 개봉일자 : 4월 5일구매사유 : 기존에 쓰던 블...,"디자인: 아주 마음에 들어요,착용감: 보통이에요,음질: 아주 만족해요,가성비: 성능..."
1,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화이트",저가전리,5,전직 컴수리기사의 버즈fe 테스트 결과보고서,주 문 : 24. 3. 27도 착 : 24. 3. 28 제 품 : 갤럭시버즈F...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 보통이에요,가성비: 성능에..."
2,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",원*,5,가성비 그자체의 무선 이어폰입니다,내돈내산 솔직후기!!!----------------------------------...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성..."
3,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",유*상,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품,작년 겨울에 쿠팡에서 산 저렴한 블루투스 이어폰이 반년만에 사망해서 버즈FE로 넘어...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 적..."
4,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화이트",Intp개발자,5,"""화이트 갤럭시 버즈 FE, 내돈내산! 최고!","(づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화...","디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성..."


In [5]:
# 리뷰 제목과 내용 모두 리뷰이기에 합쳐준다.
df['review'] = df['리뷰 제목'] + df['리뷰 내용']
df

,상품명,구매자 이름,구매자 평점,리뷰 제목,리뷰 내용,맛 만족도,review
0,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",이*영,5,"가성비 갑, 성능 좋아요!! 추천드립니다!",**내돈내산 후기입니다**구매 및 개봉일자 : 4월 5일구매사유 : 기존에 쓰던 블...,"디자인: 아주 마음에 들어요,착용감: 보통이에요,음질: 아주 만족해요,가성비: 성능...","가성비 갑, 성능 좋아요!! 추천드립니다!**내돈내산 후기입니다**구매 및 개봉일자..."
1,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화이트",저가전리,5,전직 컴수리기사의 버즈fe 테스트 결과보고서,주 문 : 24. 3. 27도 착 : 24. 3. 28 제 품 : 갤럭시버즈F...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 보통이에요,가성비: 성능에...",전직 컴수리기사의 버즈fe 테스트 결과보고서주 문 : 24. 3. 27도 착 : ...
2,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",원*,5,가성비 그자체의 무선 이어폰입니다,내돈내산 솔직후기!!!----------------------------------...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성...",가성비 그자체의 무선 이어폰입니다내돈내산 솔직후기!!!----------------...
3,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",유*상,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품,작년 겨울에 쿠팡에서 산 저렴한 블루투스 이어폰이 반년만에 사망해서 버즈FE로 넘어...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 적...",러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품작년 겨울에 쿠팡에서 산 저렴한...
4,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화이트",Intp개발자,5,"""화이트 갤럭시 버즈 FE, 내돈내산! 최고!","(づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화...","디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성...","""화이트 갤럭시 버즈 FE, 내돈내산! 최고!(づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버즈..."
...,...,...,...,...,...,...,...
7095,"삼성전자 갤럭시버즈 플러스 블루투스 이어폰, SM-R175, 블랙",이*현,4,등록된 헤드라인이 없습니다,사운드 품질은 생각보다 좀 떨어지는듯 싶고 갤럭시 웨어러블 설정에서 터치패드 잠가도...,만족도 평가 없음,등록된 헤드라인이 없습니다사운드 품질은 생각보다 좀 떨어지는듯 싶고 갤럭시 웨어러블...
7096,"삼성전자 갤럭시버즈 플러스 블루투스 이어폰, SM-R175, 블루",이*지,4,만족,"다행히 페어링 불량, 오류난거 없었구요 사용 잘 하구 있습니다 박스안에 덩그러니 있...","음질: 적당히 만족해요,디자인: 마음에 들어요,착용감: 아주 좋아요,","만족다행히 페어링 불량, 오류난거 없었구요 사용 잘 하구 있습니다 박스안에 덩그러니..."
7097,"삼성전자 갤럭시버즈 플러스 블루투스 이어폰, SM-R175, 화이트",임*란,5,자주 구매할 것 같아요. 불량사항 없습니다.,아버지가 너무 좋아하시더라구요.쿠팡으로 처음 브랜드 상품을 구매했는데하자없이 안전하...,"음질: 아주 만족해요,디자인: 아주 마음에 들어요,착용감: 아주 좋아요,",자주 구매할 것 같아요. 불량사항 없습니다.아버지가 너무 좋아하시더라구요.쿠팡으로 ...
7098,"삼성전자 갤럭시버즈 플러스 블루투스 이어폰, SM-R175, 핑크",박*영,4,선으로부터의 자유...앞으로 기대됩니다~^^,색상은 예상했던대로 산뜻하니 세련되보이구요..첨 블루투스 이어폰을 사용하는거라 아직...,"디자인: 마음에 들어요,착용감: 그저 그래요,음질: 적당히 만족해요,",선으로부터의 자유...앞으로 기대됩니다~^^색상은 예상했던대로 산뜻하니 세련되보이구...


In [6]:
df = df.rename(columns = {'구매자 평점':'rating'})
df.head()

,상품명,구매자 이름,rating,리뷰 제목,리뷰 내용,맛 만족도,review
0,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",이*영,5,"가성비 갑, 성능 좋아요!! 추천드립니다!",**내돈내산 후기입니다**구매 및 개봉일자 : 4월 5일구매사유 : 기존에 쓰던 블...,"디자인: 아주 마음에 들어요,착용감: 보통이에요,음질: 아주 만족해요,가성비: 성능...","가성비 갑, 성능 좋아요!! 추천드립니다!**내돈내산 후기입니다**구매 및 개봉일자..."
1,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화이트",저가전리,5,전직 컴수리기사의 버즈fe 테스트 결과보고서,주 문 : 24. 3. 27도 착 : 24. 3. 28 제 품 : 갤럭시버즈F...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 보통이에요,가성비: 성능에...",전직 컴수리기사의 버즈fe 테스트 결과보고서주 문 : 24. 3. 27도 착 : ...
2,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",원*,5,가성비 그자체의 무선 이어폰입니다,내돈내산 솔직후기!!!----------------------------------...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성...",가성비 그자체의 무선 이어폰입니다내돈내산 솔직후기!!!----------------...
3,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 그라파이트",유*상,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품,작년 겨울에 쿠팡에서 산 저렴한 블루투스 이어폰이 반년만에 사망해서 버즈FE로 넘어...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 적...",러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품작년 겨울에 쿠팡에서 산 저렴한...
4,"삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화이트",Intp개발자,5,"""화이트 갤럭시 버즈 FE, 내돈내산! 최고!","(づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버즈 FE 블루투스 이어폰, SM-R400N, 화...","디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성...","""화이트 갤럭시 버즈 FE, 내돈내산! 최고!(づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버즈..."


In [7]:
df2 = df[['rating', 'review']]
df2

,rating,review
0,5,"가성비 갑, 성능 좋아요!! 추천드립니다!**내돈내산 후기입니다**구매 및 개봉일자..."
1,5,전직 컴수리기사의 버즈fe 테스트 결과보고서주 문 : 24. 3. 27도 착 : ...
2,5,가성비 그자체의 무선 이어폰입니다내돈내산 솔직후기!!!----------------...
3,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품작년 겨울에 쿠팡에서 산 저렴한...
4,5,"""화이트 갤럭시 버즈 FE, 내돈내산! 최고!(づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버즈..."
...,...,...
7095,4,등록된 헤드라인이 없습니다사운드 품질은 생각보다 좀 떨어지는듯 싶고 갤럭시 웨어러블...
7096,4,"만족다행히 페어링 불량, 오류난거 없었구요 사용 잘 하구 있습니다 박스안에 덩그러니..."
7097,5,자주 구매할 것 같아요. 불량사항 없습니다.아버지가 너무 좋아하시더라구요.쿠팡으로 ...
7098,4,선으로부터의 자유...앞으로 기대됩니다~^^색상은 예상했던대로 산뜻하니 세련되보이구...


In [10]:
# 리뷰 데이터만 가져오기
review = df2['review']

# 빈 리스트를 만들어 명사 토큰을 추출해 저장한다.
review_noun_tk = []
for n in review:
    #에어팟과는 달리 float형 오류가 발생해 캐스팅
    strN = str(n)
    review_noun_tk.append(okt.nouns(strN)) # 명사 형태소만 출력

In [11]:
#한글자로만 추출된 데이터를 제거하기
review_noun_tk2 = []
for d in review_noun_tk:
    item = [i for i in d if len(i) > 1] # 뽑아낸 토큰 중 크기가 1 이상만 추출
    review_noun_tk2.append(item)

## LDA 모델 구축

In [12]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

In [13]:
print(dict[1]) # 성공!

가성


In [14]:
# 단어의 빈도수를 확인하기 위해 doc2bow 로 변환한다.
# BoW 모델은 문서의 단어의 빈도수를 표현해준다.
corpus = [dict.doc2bow(word) for word in review_noun_tk2]

In [15]:
# corpus 출력을 통해 (word_id, work_count)의 BoW 구성을 확인
print(corpus)

[[(0, 3), (1, 3), (2, 2), (3, 1), (4, 1), (5, 4), (6, 2), (7, 2), (8, 2), (9, 1), (10, 4), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 11), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 4), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 4), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 5), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 2), (71, 2), (72, 1), (73, 1), (74, 1), (75, 2), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 3), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 3), (104, 1)], [(0, 4), (1, 3), (5, 4), (11, 3), (21, 2), (22, 1), (24,

In [16]:
# 토픽의 갯수 k 개 생성
k = 3

#gensim 라이브러리를 사용해 lda 모델을 생성하는 코드 각 옵션은 만들어놓은 
#corpus lda모델
#iterations 각 문서의 주제 할당을 반복하는 횟수
#num_topics 는 위에 지정한 k 값으로 토픽의 갯수
#id2word 각 토큰의 id에 해당하는 토큰을 연결해준다
#passes는 앞서 넣은 corpus를 학습 반복하는 횟수이다 수가 높아지면 더 잘 학습하지만 그만큼 시간이 증가한다.
#workers 쓰레드의 수 (작업자의 수)이다. 보통은 본인의 컴퓨터에 맞춰 설정하게 된다.
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations = 12, num_topics = k, id2word = dict, passes = 5, workers = 10)

In [17]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.064*"버즈" + 0.023*"음질" + 0.022*"프로" + 0.018*"사용" + 0.016*"이어폰" + 0.013*"착용" + 0.013*"에어팟" + 0.013*"구매" + 0.013*"갤럭시" + 0.012*"가격" + 0.012*"소리" + 0.010*"제품" + 0.009*"플러스" + 0.008*"통화" + 0.008*"라이브"'), (1, '0.027*"사용" + 0.023*"이어폰" + 0.020*"버즈" + 0.018*"음질" + 0.015*"착용" + 0.015*"구매" + 0.015*"노이즈" + 0.014*"슬링" + 0.014*"소리" + 0.012*"기능" + 0.009*"헤드라인" + 0.009*"정도" + 0.009*"제품" + 0.009*"등록" + 0.009*"귀가"'), (2, '0.061*"등록" + 0.039*"헤드라인" + 0.024*"리뷰" + 0.023*"버즈" + 0.020*"용이" + 0.019*"이어폰" + 0.016*"착용" + 0.016*"음질" + 0.012*"사용" + 0.012*"구매" + 0.012*"갤럭시" + 0.010*"제품" + 0.010*"가격" + 0.008*"삼성" + 0.008*"소리"')]


In [18]:
# lda모델을 디스플레이로 출력
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict)
pyLDAvis.display(lda_vis)